In [1]:
from depsurf import (
    diff_img_pairs,
    OUTPUT_PATH,
    VERSIONS_ARCH,
    VERSIONS_FLAVOR,
    VERSION_54,
)


results = []

for group, versions in [
    ("arch", VERSIONS_ARCH),
    ("flavor", VERSIONS_FLAVOR),
]:
    pairs = [(VERSION_54, v) for v in versions]
    results.append(diff_img_pairs(group, pairs, result_path=OUTPUT_PATH / group))

[     deps.py:188] INFO: Diffing arch
[     deps.py:194] INFO: Comparing 5.4.0-26-generic-amd64 and 5.4.0-26-generic-arm64 to /Users/szhong/Downloads/bpf-study/btf/output/arch/5.4.0-26-generic-amd64_5.4.0-26-generic-arm64
[     deps.py:194] INFO: Comparing 5.4.0-26-generic-amd64 and 5.4.0-26-generic-arm64 to /Users/szhong/Downloads/bpf-study/btf/output/arch/5.4.0-26-generic-amd64_5.4.0-26-generic-arm64
[     deps.py:194] INFO: Comparing 5.4.0-26-generic-amd64 and 5.4.0-26-generic-armhf to /Users/szhong/Downloads/bpf-study/btf/output/arch/5.4.0-26-generic-amd64_5.4.0-26-generic-armhf
[     deps.py:194] INFO: Comparing 5.4.0-26-generic-amd64 and 5.4.0-26-generic-ppc64el to /Users/szhong/Downloads/bpf-study/btf/output/arch/5.4.0-26-generic-amd64_5.4.0-26-generic-ppc64el
[     deps.py:194] INFO: Comparing 5.4.0-26-generic-amd64 and 5.4.0-26-generic-s390x to /Users/szhong/Downloads/bpf-study/btf/output/arch/5.4.0-26-generic-amd64_5.4.0-26-generic-s390x
[     deps.py:188] INFO: Diffing flavo

In [2]:
import pandas as pd
from depsurf.output import save_df

df = pd.concat(results, axis=1)
df = df.loc[(df != 0).any(axis=1)]
df = df.fillna(0).astype(int)
save_df(df, "config")

[       pd.py:30 ] INFO: Saved dataframe to /Users/szhong/Downloads/bpf-study/btf/output/config.pkl
[       pd.py:34 ] INFO: Saved dataframe to /Users/szhong/Downloads/bpf-study/btf/output/config.txt


arch                                                                                                                                                                                                   flavor                                                                                                                                                                                                 
                               (5.4.0-26-generic-amd64, 5.4.0-26-generic-arm64) (5.4.0-26-generic-amd64, 5.4.0-26-generic-armhf) (5.4.0-26-generic-amd64, 5.4.0-26-generic-ppc64el) (5.4.0-26-generic-amd64, 5.4.0-26-generic-s390x) (5.4.0-26-generic-amd64, 5.4.0-26-lowlatency-amd64) (5.4.0-26-generic-amd64, 5.4.0-1009-aws-amd64) (5.4.0-26-generic-amd64, 5.4.0-1009-gcp-amd64) (5.4.0-26-generic-amd64, 5.4.0-1009-oracle-amd64) (5.4.0-26-generic-amd64, 5.4.0-1010-azure-amd64)
Function   Old                                                            48018                                            48018                                              48018                                            48018                                               48018                                          48018                                          48018                                             48018                                            48018
           Added                                                           9216                                            12602                                               5444                                             3892                                                  57                                            328                                            450                                              1594                                              992
           Removed                                                         7872                                            11775                                              10636                                            18750                                                  41                                           1823                                            319                                               317                                             3527
           Return type changed                                                9                                               10                                                 24                                                7                                                   0                                              0                                              0                                                 0                                                1
           Param added                                                       89                                               66                                                134                                               95                                                   0                                              2                                              1                                                 1                                               13
           Param removed                                                     92                                               71                                                131                                              102                                                   0                                              2                                              1                                                 1                                               13
           Param type changed                                                61                                               58                                                 61                                               10                                                   0   

In [3]:
import pandas as pd
from depsurf import OUTPUT_PATH, DepKind

kinds = [DepKind.STRUCT, DepKind.FUNC, DepKind.TRACEPOINT, DepKind.LSM]

In [4]:
df = pd.read_pickle(OUTPUT_PATH / "config.pkl")
df = df.astype(object)


for kind in kinds:
    for row in df.loc[kind].index:
        if row == "Old":
            continue
        for col in df.columns:
            val = int(df.loc[(kind, row), col].iloc[0])
            old = int(df.loc[(kind, "Old"), col].iloc[0])
            percentage = val / old * 100
            df.loc[(kind, row), col] = f"\\val{{{val}}}{{{percentage:4.1f}}}"

for kind in kinds:
    df.loc[(kind, r"\Delta"), :] = df.loc[
        [
            (k, r)
            for k, r in df.index
            if k == kind and r not in ["Added", "Removed", "Old"]
        ],
        :,
    ].sum()


df = df.sort_index()
df.columns = [
    "".join(v2.split("-")[2:]).removeprefix("generic").removesuffix("amd64")
    for (group, (v1, v2)) in df.columns
]
df = df.drop(
    index=[
        i
        for i in df.index.get_level_values(1).unique()
        if i not in ["Added", "Removed", r"\Delta"]
    ],
    level=1,
)

df.to_latex("output/diff.tex")

df

arm64                                                                  armhf                                                                   ppc64el                                                                  s390x                                                         lowlatency                                                                aws                                                                gcp                                                             oracle                                                                azure
Function   Added                                                        \val{9216}{19.2}                                                      \val{12602}{26.2}                                                          \val{5444}{11.3}                                                       \val{3892}{ 8.1}                                                     \val{57}{ 0.1}                                                    \val{328}{ 0.7}                                                    \val{450}{ 0.9}                                                   \val{1594}{ 3.3}                                                      \val{992}{ 2.1}
           Removed                                                      \val{7872}{16.4}                                                      \val{11775}{24.5}                                                         \val{10636}{22.2}                                                      \val{18750}{39.0}                                                     \val{41}{ 0.1}                                                   \val{1823}{ 3.8}                                                    \val{319}{ 0.7}                                                    \val{317}{ 0.7}                                                     \val{3527}{ 7.3}
           \Delta   \val{9}{ 0.0}\val{89}{ 0.2}\val{92}{ 0.2}\val{61}{ 0.1}\val{3}{ 0.0}  \val{10}{ 0.0}\val{66}{ 0.1}\val{71}{ 0.1}\val{58}{ 0.1}\val{8}{ 0.0}  \val{24}{ 0.0}\val{134}{ 0.3}\val{131}{ 0.3}\val{61}{ 0.1}\val{12}{ 0.0}  \val{7}{ 0.0}\val{95}{ 0.2}\val{102}{ 0.2}\val{10}{ 0.0}\val{8}{ 0.0}  \val{0}{ 0.0}\val{0}{ 0.0}\val{0}{ 0.0}\val{0}{ 0.0}\val{0}{ 0.0}  \val{0}{ 0.0}\val{2}{ 0.0}\val{2}{ 0.0}\val{1}{ 0.0}\val{0}{ 0.0}  \val{0}{ 0.0}\val{1}{ 0.0}\val{1}{ 0.0}\val{0}{ 0.0}\val{0}{ 0.0}  \val{0}{ 0.0}\val{1}{ 0.0}\val{1}{ 0.0}\val{0}{ 0.0}\val{0}{ 0.0}  \val{1}{ 0.0}\val{13}{ 0.0}\val{13}{ 0.0}\val{3}{ 0.0}\val{2}{ 0.0}
LSM        \Delta                                                                      0                                                                      0                                                                         0                                                                      0                                                                  0                                                                  0                                                                  0                                                                  0                                                                    0
Struct     Added                                                        \val{1659}{19.7}                                                       \val{1980}{23.5}                                                           \val{570}{ 6.8}                                                        \val{694}{ 8.2}                                                      \val{4}{ 0.0}                                                     \val{83}{ 1.0}                                                     \val{68}{ 0.8}                                                    \val{474}{ 5.6}                                                      \val{257}{ 3.1}
           Removed                                                      \val{1006}{11.9}                                                       \val{1851}{22.0}                                                          \val{1624}{19.

In [5]:
df = pd.read_pickle(OUTPUT_PATH / "config.pkl")
df = df.astype(object)


for kind in kinds:
    for row in df.loc[kind].index:
        if row == "Old":
            continue
        for col in df.columns:
            val = int(df.loc[(kind, row), col].iloc[0])
            old = int(df.loc[(kind, "Old"), col].iloc[0])
            percentage = val / old * 100
            df.loc[(kind, row), col] = f"\\val{{{val}}}{{{percentage:4.1f}}}"

df.columns = [
    "".join(v2.split("-")[2:]).removeprefix("generic").removesuffix("amd64")
    for (group, (v1, v2)) in df.columns
]
df = df.drop(index="Old", level=1)

df.to_latex("output/diff.tex")

df

arm64              armhf            ppc64el              s390x      lowlatency               aws              gcp            oracle             azure
Function   Added                \val{9216}{19.2}  \val{12602}{26.2}   \val{5444}{11.3}   \val{3892}{ 8.1}  \val{57}{ 0.1}   \val{328}{ 0.7}  \val{450}{ 0.9}  \val{1594}{ 3.3}   \val{992}{ 2.1}
           Removed              \val{7872}{16.4}  \val{11775}{24.5}  \val{10636}{22.2}  \val{18750}{39.0}  \val{41}{ 0.1}  \val{1823}{ 3.8}  \val{319}{ 0.7}   \val{317}{ 0.7}  \val{3527}{ 7.3}
           Return type changed     \val{9}{ 0.0}     \val{10}{ 0.0}     \val{24}{ 0.0}      \val{7}{ 0.0}   \val{0}{ 0.0}     \val{0}{ 0.0}    \val{0}{ 0.0}     \val{0}{ 0.0}     \val{1}{ 0.0}
           Param added            \val{89}{ 0.2}     \val{66}{ 0.1}    \val{134}{ 0.3}     \val{95}{ 0.2}   \val{0}{ 0.0}     \val{2}{ 0.0}    \val{1}{ 0.0}     \val{1}{ 0.0}    \val{13}{ 0.0}
           Param removed          \val{92}{ 0.2}     \val{71}{ 0.1}    \val{131}{ 0.3}    \val{102}{ 0.2}   \val{0}{ 0.0}     \val{2}{ 0.0}    \val{1}{ 0.0}     \val{1}{ 0.0}    \val{13}{ 0.0}
           Param type changed     \val{61}{ 0.1}     \val{58}{ 0.1}     \val{61}{ 0.1}     \val{10}{ 0.0}   \val{0}{ 0.0}     \val{1}{ 0.0}    \val{0}{ 0.0}     \val{0}{ 0.0}     \val{3}{ 0.0}
           Param reordered         \val{3}{ 0.0}      \val{8}{ 0.0}     \val{12}{ 0.0}      \val{8}{ 0.0}   \val{0}{ 0.0}     \val{0}{ 0.0}    \val{0}{ 0.0}     \val{0}{ 0.0}     \val{2}{ 0.0}
Struct     Added                \val{1659}{19.7}   \val{1980}{23.5}    \val{570}{ 6.8}    \val{694}{ 8.2}   \val{4}{ 0.0}    \val{83}{ 1.0}   \val{68}{ 0.8}   \val{474}{ 5.6}   \val{257}{ 3.1}
           Removed              \val{1006}{11.9}   \val{1851}{22.0}   \val{1624}{19.3}   \val{3064}{36.4}   \val{1}{ 0.0}   \val{483}{ 5.7}  \val{123}{ 1.5}   \val{160}{ 1.9}   \val{833}{ 9.9}
           Field removed         \val{146}{ 1.7}    \val{246}{ 2.9}    \val{201}{ 2.4}    \val{325}{ 3.9}   \val{9}{ 0.1}    \val{39}{ 0.5}   \val{21}{ 0.2}     \val{9}{ 0.1}   \val{128}{ 1.5}
           Field added           \val{109}{ 1.3}    \val{119}{ 1.4}    \val{192}{ 2.3}    \val{116}{ 1.4}   \val{7}{ 0.1}     \val{9}{ 0.1}    \val{2}{ 0.0}     \val{2}{ 0.0}    \val{34}{ 0.4}
           Field type changed     \val{87}{ 1.0}    \val{165}{ 2.0}    \val{148}{ 1.8}    \val{119}{ 1.4}   \val{0}{ 0.0}     \val{1}{ 0.0}    \val{2}{ 0.0}     \val{1}{ 0.0}     \val{8}{ 0.1}
Tracepoint Added                  \val{45}{ 6.0}     \val{70}{ 9.3}     \val{25}{ 3.3}     \val{61}{ 8.1}   \val{0}{ 0.0}     \val{4}{ 0.5}    \val{0}{ 0.0}     \val{9}{ 1.2}    \val{26}{ 3.5}
           Removed               \val{112}{14.9}    \val{132}{17.6}    \val{129}{17.2}    \val{254}{33.8}   \val{0}{ 0.0}     \val{9}{ 1.2}    \val{0}{ 0.0}     \val{0}{ 0.0}    \val{39}{ 5.2}

In [7]:
# df = pd.read_pickle(OUTPUT_PATH / "config.pkl")

# for kind in kinds:
#     df.loc[(kind, r"\sum"), :] = df.loc[(kind, "Old"), :] + df.loc[(kind, "Added"), :] - df.loc[(kind, "Removed"), :]

# df

# df.to_latex("output/diff.tex")